# PySAP
## 1. Dimensions

In [13]:
ground_floor_area = 40
first_floor_area = 40
ground_floor_storey_height = 2.5

first_floor_storey_height = ground_floor_storey_height + 0.25 # RdSAP
TFA = ground_floor_area + first_floor_area
ground_V = ground_floor_area * ground_floor_storey_height
first_V = first_floor_area * first_floor_storey_height
DV = ground_V + first_V

print(f"Box 4 TFA: {TFA}")
print(f"Box 5 Dwelling volume: {DV}")

Box 4 TFA: 80
Box 5 Dwelling volume: 210.0


## 2. Ventilation rate

In [14]:
open_chimneys = 0 * 80
open_flues = 0 * 20
closed_fire = 0 * 10 
solid_fuel = 0 * 20 
other_heater = 0 * 35 
blocked = 0 * 20  
fans = 0 * 10  
vents = 0 * 10 
flueless = 0 * 40

has_test = False
sheltered_sides = 2 
system = 'natural'

infiltration = (
    open_chimneys
    + open_flues
    + closed_fire
    + solid_fuel
    + other_heater
    + blocked
    + fans
    + vents
    + flueless
    )
infiltration_rate = infiltration / DV

if has_test:
    if ap50 > 0:
        adjusted_infiltration = (ap50 / 20) + infiltration_rate
    else:
        adjusted_infiltration = (0.263 * (ap4 ** 0.924)) + infiltration_rate
else:
    # Manual infiltration
    storeys = 2
    construction = 'masonry'
    floor = 'unsealed'
    lobby = 0 # RdSAP - House, bungalow or park home: no
    proofing = 0.05 # If no draught lobby, enter 0.05, else enter 0
    additional = (storeys - 1) * 0.1 if storeys > 1 else 0
    structural = 0.25 if construction in ['steel', 'timber'] else 0.35

    # RdSAP - Age band of main dwelling A to E: unsealed. Age band of main dwelling F to L: sealed
    floor_infiltration = 0.2 if floor == 'unsealed' else 0.1 if floor == 'sealed' else 0
    lobby_infiltration = 0.05 if lobby == 0 else 0
    window_infiltration = 0.25 - (0.2 * proofing / 100)
    adjusted_infiltration = (
        infiltration_rate
        + additional
        + structural
        + floor_infiltration
        + lobby_infiltration
        + window_infiltration
        )

shelter_factor = 1 - (0.075 * sheltered_sides)
final_infiltration = adjusted_infiltration * shelter_factor

# Monthly wind factor
monthly_wind = [5.10, 5.00, 4.90, 4.40, 4.30, 3.80, 3.80, 3.70, 4.00, 4.30, 4.50, 4.70]
wind_factors = [w/4 for w in monthly_wind]
adjusted_monthly = [final_infiltration * wf for wf in wind_factors]

# Mechanical ventilation
effective_ach = []
for monthly in adjusted_monthly:
    if system == 'mvhr':
        efficiency = 80
        ach = monthly + 0.5 * (1 - efficiency/100)
    elif system == 'balanced':
        ach = monthly + 0.5
    elif system == 'mechanical':
        ach = monthly + 0.5 if monthly < 0.25 else monthly + 0.25
    else:
        # natural
        ach = monthly if monthly >= 1 else 0.5 + (0.5 * monthly**2)
    effective_ach.append(ach)

# Results
print(f"Box 8 Initial infiltration rate: {infiltration_rate:.2f} ACH")
print(f"Box 18 Adjusted infiltration: {adjusted_infiltration:.2f} ACH")
print(f"Box 21 With shelter factor: {final_infiltration:.2f} ACH")
print(f"Box 25 Effective ach: {[round(x, 2) for x in effective_ach]} ACH")

Box 8 Initial infiltration rate: 0.00 ACH
Box 18 Adjusted infiltration: 0.95 ACH
Box 21 With shelter factor: 0.81 ACH
Box 25 Effective ach: [1.03, 1.01, 0.99, 0.89, 0.88, 0.79, 0.79, 0.78, 0.83, 0.88, 0.91, 0.95] ACH
